In [20]:
import DataRetriever as dr
import pandas as pd

# I had to load it in like this to get the timestamp index to become a TimeStampIndex... ... ...
year_2 = pd.read_csv("C:/Users/madsc/OneDrive - Aalborg Universitet/P6/Data/raw_data/All-Subsystems-minute-Year2.csv",
                     parse_dates=["Timestamp"], infer_datetime_format=True, index_col=["Timestamp"])

In [21]:
# Set the index to be Datetime UTC
year_2.index = pd.to_datetime(year_2.index, utc=True)

In [24]:
del year_2["DayOfWeek"]  # Remove the non-number attribute to enable the next two lines
year_2[year_2 > 0] = 1
year_2[year_2 <= 0] = 0

In [25]:
# Resample the dataset, calculating the mean for each hour yielding real values in [0, 1]
year_2 = year_2.resample(rule="H").mean()

In [26]:
# Rounding it to the nearest number 0 or 1
year_2 = year_2.round()

In [ ]:
# TODO: Find out how to make association rules based on this dataset. Get it on the same format as in the DM course?
year_2